
#[Colab 환경설정]



In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 2.3 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 5.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595756 sha256=157bf1e9b64b831e7b398615dd55fb41d78f11959d6e821eac2e6ea1da60cc27
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 5.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-f7pis55f
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-f7pis55f
     |████████████████████████████████| 132 kB 5.4 MB/s 
     |████████████████████████████████| 4.5 MB 38.9 MB/s 
     |████████████████████████████████| 79 kB 9.6 MB/s 
     |████████████████████████████████| 8.8 MB 49.9 MB/s 
     |████████████████████████████████| 127 kB 58.3 MB/s 
  Created wheel for kobert: filename=kobert-0.2.3-py3-none-any.whl size=15674 sha256=1c5c1a8438da60693c13c7c052d23828b9e57393234462e29c14714482a0d36a
  Stored in directory: /tmp/pip-ephem-wheel-cache-kjvbg2lf/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urll

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook


In [ ]:
#kobert
from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer


#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_excel('/content/drive/MyDrive/df_data.xlsx')

In [ ]:
data

,sentence,label
0,오늘 반장 선거에서 내가 반장이 됐어! 친구들이 날 믿어줘서 너무 고맙다.,0
1,오늘 시험에 친구가 알려준 문제가 나왔어. 결과가 좋아서 기쁘다.,0
2,오늘 대학 합격 통지를 받았어. 엄마가 너무 기뻐하는데 나는 눈물이 났어.,0
3,오늘 졸업식인데 선생님께서 아프셔서 오지 못하셨어. 그동안 감사하다는 말도 못했는데.,0
4,나 올림피아드에서 금상 받았어! 그동안 고생이 헛되지 않아서 너무 기뻐.,0
...,...,...
12394,주변인들에게 안부전화를 해봐야겠어,5
12395,이젠 운전하지 말아야겠어,5
12396,비상약을 차에 뒀는데 어디에 간거지?,5
12397,손이 아프긴 했지만 부러진거라고?,5


In [ ]:
data.to_csv('/content/model_data.csv',header=True,index=False, encoding='utf-8-sig')

In [ ]:
shuffled_data = data.sample(frac=1).reset_index(drop=True)
shuffled_data

,sentence,label
0,오늘 내 혈압이 떨어졌어. 정말 다행이야.,0
1,프로젝트 실적이 성공적이었어. 느긋한 마음으로 업무에 집중할 수 있을 것 같아.,0
2,언젠가부터 내 지갑에 돈이 없어지기 시작했는데 그게 아들 때문이었어.,1
3,남편과 자식들이 우울증 잘 극복하라고 재미있는 이벤트를 해줘서 기뻐.,0
4,우리 개가 죽어서 슬퍼.,5
...,...,...
12394,일찍 자고 일찍 일어났더니 공부할 때 마음이 한결 편안해.,0
12395,오늘 내가 주번은 아니었는데 몸이 안 좋은 친구들을 대신해서 청소를 해줬어.,4
12396,오늘 회식에서 술을 입에도 대지 않았어. 불안해서.,3
12397,딸이 성적이 왜 이 모양인지.,2


In [ ]:
shuffled_data.label = shuffled_data.label.astype(int) 
shuffled_data.sentence = shuffled_data.sentence.astype(str) 
shuffled_data

,sentence,label
0,오늘 내 혈압이 떨어졌어. 정말 다행이야.,0
1,프로젝트 실적이 성공적이었어. 느긋한 마음으로 업무에 집중할 수 있을 것 같아.,0
2,언젠가부터 내 지갑에 돈이 없어지기 시작했는데 그게 아들 때문이었어.,1
3,남편과 자식들이 우울증 잘 극복하라고 재미있는 이벤트를 해줘서 기뻐.,0
4,우리 개가 죽어서 슬퍼.,5
...,...,...
12394,일찍 자고 일찍 일어났더니 공부할 때 마음이 한결 편안해.,0
12395,오늘 내가 주번은 아니었는데 몸이 안 좋은 친구들을 대신해서 청소를 해줬어.,4
12396,오늘 회식에서 술을 입에도 대지 않았어. 불안해서.,3
12397,딸이 성적이 왜 이 모양인지.,2


In [ ]:
Data = []
for q, label in zip(shuffled_data['sentence'], shuffled_data['label'])  :
  if(len(str(q))<100):
    data2 = []
    data2.append(q)
    data2.append(str(label))

    Data.append(data2)



In [ ]:
Data

[['오늘 내 혈압이 떨어졌어. 정말 다행이야.', '0'],
 ['프로젝트 실적이 성공적이었어. 느긋한 마음으로 업무에 집중할 수 있을 것 같아.', '0'],
 ['언젠가부터 내 지갑에 돈이 없어지기 시작했는데 그게 아들 때문이었어.', '1'],
 ['남편과 자식들이 우울증 잘 극복하라고 재미있는 이벤트를 해줘서 기뻐.', '0'],
 ['우리 개가 죽어서 슬퍼.', '5'],
 ['와 씨발 상상불허', '6'],
 ['민주당이 압승인데 왜 이런걸로 지랄인지....', '6'],
 ['내 건강을 생각해서 딸과 사위가 나와 함께 살겠다고 하네. 애들한테 너무 고맙다.', '0'],
 ['찐따는 ㅇㅂ ', '6'],
 ['은퇴한 나이지만 만성질환도 없어 나는.', '0'],
 ['두 달 동안 준비한 프러포즈를 거절당했어. 그래서 부끄러워.', '5'],
 ['미스터리의 풀이를 해냅니다.', '0'],
 ['남편이 조기축구하다가 무릎이 다쳐서 일을 못나가. 내가 아주 미쳐.', '2'],
 ['이번에 건강검진에서 유방암 검사를 받았는데 정밀검사가 필요하다고 하는데 걱정이야.', '5'],
 ['혈압이 이제 정상이래. 너무 좋아.', '0'],
 ['우리 부모님께서 오늘 우리 반에 햄버거를 돌리셔서 어깨가 으쓱했어.', '0'],
 ['지랄 개소리 쳐하고 자빠졌네 보지년아', '6'],
 ['나는 염세적인 생각을 하는 사람은 건강하게 살지 못할 거 같아.', '1'],
 ['내 졸업식에도 누나 졸업식 때처럼 가족들이 다 와줬으면 좋겠는데 가능할까?', '4'],
 ['세상에 내 비밀을 털어놓을 사람이 아무도 없는 것처럼 느껴져. 제일 친한 친구한테 상처 받았어.', '4'],
 ['내가 지원했던 회사마다 다 날 탈락시켰는데 이유를 모르겠어.', '2'],
 ['정말 갖고싶은게 있었는데 남편의 반대가 너무 심해서 못사게됬어', '4'],
 ['부랄 달랑 시발년들아 ', '6'],
 ['은퇴하고 몸이 아파서 집에만 있어.', '4'],
 ['엄마와 아빠 사이에서 내

#[Train data & Test Data]

In [ ]:
import re

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(Data, test_size=0.3, shuffle=False, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

8562
3670


In [ ]:
dataset_train[0]

['오늘 내 혈압이 떨어졌어. 정말 다행이야.', '0']

#[BERTDataset]

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 6
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
from transformers import BertModel

tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
dataset_train[0]

['오늘 내 혈압이 떨어졌어. 정말 다행이야.', '0']

In [ ]:
data_train[0]

(array([   2, 3419, 1434, 5069, 6825, 7096, 1856, 6855, 7248, 6855,  517,
          54, 4102, 1562, 7881, 7096, 6844,  517,   54,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(20, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 0)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device) #gpu

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
train_dataloader

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/134 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.008787155151367 train acc 0.078125
epoch 1 train acc 0.3455457089552239


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 5 test acc 0.6777037617554859


  0%|          | 0/134 [00:00<?, ?it/s]

epoch 6 batch id 1 loss 0.4373185634613037 train acc 0.890625
epoch 6 train acc 0.8724067164179105


  0%|          | 0/58 [00:00<?, ?it/s]

epoch 6 test acc 0.6884306426332288


#[테스트부분]





In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 1:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("불안이")
            elif np.argmax(logits) == 4:
                test_eval.append("상처가")
            elif np.argmax(logits) == 5:
                test_eval.append("당황이")
            elif np.argmax(logits) == 6:
                test_eval.append("욕설이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

         

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 1987은 진짜  너무 슬펐어요


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 전두환은 진짜 희대의 살인마라고 밖에 말 못한다. 
>> 입력하신 내용에서 상처가 느껴집니다.


하고싶은 말을 입력해주세요 : 1987이 역대급이지.. 진짜 저장면은.. 미친듯이 화도 나고 눈물도 난다.
>> 입력하신 내용에서 욕설이 느껴집니다.


하고싶은 말을 입력해주세요 : 1987은 정말 감정선이 고스란히 전달이 되서 지금도 눈물이 비오듯 쏟아진다
>> 입력하신 내용에서 슬픔이 느껴집니다.




KeyboardInterrupt: ignored

In [ ]:
torch.save(model, 'ko_bert.pt')

In [ ]:
model1 = torch.load('/content/ko_bert.pt')
model1.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁨이")
            elif np.argmax(logits) == 1:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("불안이")
            elif np.argmax(logits) == 4:
                test_eval.append("상처가")
            elif np.argmax(logits) == 5:
                test_eval.append("당황이")
            elif np.argmax(logits) == 6:
                test_eval.append("욕설이")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")


using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 철수가 정말 걱정이돼


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 상처가 느껴집니다.


하고싶은 말을 입력해주세요 : 너무 불안해
>> 입력하신 내용에서 불안이 느껴집니다.


하고싶은 말을 입력해주세요 : 행복해
>> 입력하신 내용에서 기쁨이 느껴집니다.




KeyboardInterrupt: ignored